## Bibliotecas

In [1]:
import sys
sys.path.append('../../../')

In [2]:
## Spark SQL
from pyspark.sql import SparkSession
import pyspark.sql.functions as f

# Spark ML
from pyspark.ml.pipeline import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [3]:
spark = (
    SparkSession
    .builder
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
    .config("spark.driver.memory", "6g")
    .getOrCreate()
)

## Spark ML

### Binary Classification

#### Data Split

In [4]:
df_train = spark.read.parquet('../../../data/raw/raw_train')
df_test = spark.read.parquet('../../../data/raw/raw_test')

#### Preprocessing

In [5]:
from src.ml.preprocessing.preprocessing import SparkPreprocessor

In [6]:
from src.ml.preprocessing.normalizer import SparkScaler

In [15]:
preproc = SparkPreprocessor({'zscore': 'Age'}, ['Sex', 'Pclass'])

In [16]:
df_preproc = preproc.execute(df_train, )

INFO:root:Normalizing


,Survived,Pclass,Sex,Age,Sex_indexed,Pclass_indexed,Sex_ohe,Pclass_ohe,zscore_vec,zscore_scaled,features
0,1,2,female,34.0,1.0,2.0,"(0.0, 1.0)","(0.0, 0.0, 1.0)",[34.0],[2.3748458010440516],"[0.0, 1.0, 0.0, 0.0, 1.0, 2.3748458010440516]"
1,1,2,female,31.0,1.0,2.0,"(0.0, 1.0)","(0.0, 0.0, 1.0)",[31.0],[2.1653005833048704],"[0.0, 1.0, 0.0, 0.0, 1.0, 2.1653005833048704]"
2,1,1,male,36.0,0.0,1.0,"(1.0, 0.0)","(0.0, 1.0, 0.0)",[36.0],[2.514542612870172],"[1.0, 0.0, 0.0, 1.0, 0.0, 2.514542612870172]"
3,1,3,male,29.0,0.0,0.0,"(1.0, 0.0)","(1.0, 0.0, 0.0)",[29.0],[2.0256037714787496],"[1.0, 0.0, 1.0, 0.0, 0.0, 2.0256037714787496]"
4,0,2,male,18.0,0.0,2.0,"(1.0, 0.0)","(0.0, 0.0, 1.0)",[18.0],[1.257271306435086],"[1.0, 0.0, 0.0, 0.0, 1.0, 1.257271306435086]"
...,...,...,...,...,...,...,...,...,...,...,...
487,1,2,female,34.0,1.0,2.0,"(0.0, 1.0)","(0.0, 0.0, 1.0)",[34.0],[2.3748458010440516],"[0.0, 1.0, 0.0, 0.0, 1.0, 2.3748458010440516]"
488,1,2,female,30.0,1.0,2.0,"(0.0, 1.0)","(0.0, 0.0, 1.0)",[30.0],[2.09545217739181],"[0.0, 1.0, 0.0, 0.0, 1.0, 2.09545217739181]"
489,0,3,male,32.0,0.0,0.0,"(1.0, 0.0)","(1.0, 0.0, 0.0)",[32.0],[2.235148989217931],"[1.0, 0.0, 1.0, 0.0, 0.0, 2.235148989217931]"
490,0,3,male,30.0,0.0,0.0,"(1.0, 0.0)","(1.0, 0.0, 0.0)",[30.0],[2.09545217739181],"[1.0, 0.0, 1.0, 0.0, 0.0, 2.09545217739181]"


#### Model

In [17]:
from src.ml.model.trainer import SparkTrainer

Could not import lightgbm, required if using LGBMExplainableModel


In [11]:
model = SparkTrainer()

In [12]:
model.train(df_train_preproc, True, LogisticRegression, labelCol='Survived', family='binomial')

Confusion Matrix
+-------+---+---+
|Outcome|  0|  1|
+-------+---+---+
|      0| 42|  8|
|      1|  3| 31|
+-------+---+---+


Results
+------------+-----------+
|   Accuracy |   ROC AUC |
+============+===========+
|    3.31818 |  0.934118 |
+------------+-----------+

+-----------+-------------+----------+----------+
|   Outcome |   Precision |   Recall |       F1 |
+===========+=============+==========+==========+
|         0 |    0.933333 | 0.84     | 0.884211 |
+-----------+-------------+----------+----------+
|         1 |    0.794872 | 0.911765 | 0.849315 |
+-----------+-------------+----------+----------+


TypeError: __init__() takes from 1 to 4 positional arguments but 5 were given